In [1]:
import pyvista as pv
sphere = pv.Sphere()

# short example
sphere.plot(jupyter_backend='trame')

# # long example
# plotter = pv.Plotter(notebook=True)
# plotter.add_mesh(sphere)
# plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:38655/index.html?ui=P_0x74d4c84030e0_0&reconnect=auto" class="pyvi…

In [2]:
import numpy as np
from mpi4py import MPI


import dolfinx.plot as plot
from dolfinx.fem import Function, functionspace
from dolfinx.mesh import CellType, compute_midpoints, create_unit_cube, create_unit_square, meshtags

try:
    import pyvista
except ModuleNotFoundError:
    print("pyvista is required for this demo")
    exit(0)

# If environment variable PYVISTA_OFF_SCREEN is set to true save a png
# otherwise create interactive plot

# Set some global options for all plots
transparent = False
figsize = 800


In [3]:

def plot_scalar():
    # We start by creating a unit square mesh and interpolating a
    # function into a degree 1 Lagrange space
    msh = create_unit_square(
        MPI.COMM_WORLD, 12, 12, cell_type=CellType.quadrilateral, dtype=np.float64
    )
    V = functionspace(msh, ("Lagrange", 1))
    u = Function(V, dtype=np.float64)
    u.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(2 * x[1] * np.pi))
    # To visualize the function u, we create a VTK-compatible grid to
    # values of u to
    cells, types, x = plot.vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = u.x.array
    print(u.x.array)

    # The function "u" is set as the active scalar for the mesh, and
    # warp in z-direction is set
    grid.set_active_scalars("u")
    warped = grid.warp_by_scalar()

    # A plotting window is created with two sub-plots, one of the scalar
    # values and the other of the mesh is warped by the scalar values in
    # z-direction
    subplotter = pyvista.Plotter(shape=(1, 2))
    subplotter.subplot(0, 0)
    subplotter.add_text("Scalar contour field", font_size=14, color="black", position="upper_edge")
    subplotter.add_mesh(grid, show_edges=True, show_scalar_bar=True)
    subplotter.view_xy()

    subplotter.subplot(0, 1)
    subplotter.add_text("Warped function", position="upper_edge", font_size=14, color="black")
    sargs = dict(
        height=0.8,
        width=0.1,
        vertical=True,
        position_x=0.05,
        position_y=0.05,
        fmt="%1.2e",
        title_font_size=40,
        color="black",
        label_font_size=25,
    )
    subplotter.set_position([-3, 2.6, 0.3])
    subplotter.set_focus([3, -1, -0.15])
    subplotter.set_viewup([0, 0, 1])
    subplotter.add_mesh(warped, show_edges=True, scalar_bar_args=sargs)
    if pyvista.OFF_SCREEN:
        subplotter.screenshot(
            "2D_function_warp.png",
            transparent_background=transparent,
            window_size=[figsize, figsize],
        )
    else:
        subplotter.show()

In [4]:
plot_scalar()

[ 4.22403703e-34  7.26639270e-18  7.52272328e-18  1.29409523e-01
  1.25857614e-17  2.24143868e-01  1.45327854e-17  2.50000000e-01
  1.45327854e-17  2.58819045e-01  4.33012702e-01  2.05524622e-17
  3.53553391e-01  1.25857614e-17  2.24143868e-01  5.00000000e-01
  6.12372436e-01  2.51715227e-17  4.33012702e-01  7.26639270e-18
  1.29409523e-01  4.33012702e-01  7.07106781e-01  7.50000000e-01
  2.80751855e-17  4.82962913e-01  1.77975291e-33  3.16961915e-17
  2.50000000e-01  6.12372436e-01  8.66025404e-01  8.36516304e-01
  2.90655708e-17  5.00000000e-01 -7.26639270e-18 -1.29409523e-01
  6.12323400e-17  3.53553391e-01  7.50000000e-01  9.65925826e-01
  8.66025404e-01  2.80751855e-17  4.82962913e-01 -1.25857614e-17
 -2.24143868e-01 -2.50000000e-01  8.65956056e-17  4.33012702e-01
  8.36516304e-01  1.00000000e+00  8.36516304e-01  2.51715227e-17
  4.33012702e-01 -1.45327854e-17 -2.58819045e-01 -4.33012702e-01
 -3.53553391e-01  1.06057524e-16  4.82962913e-01  8.66025404e-01
  9.65925826e-01  7.50000

Widget(value='<iframe src="http://localhost:38655/index.html?ui=P_0x74d4612f3c50_1&reconnect=auto" class="pyvi…